In [1]:
import numpy as np
import pandas as pd

# ML-mетрики качества

### Hit rate

Hit rate = был ли хотя бы 1 релевантный товар среди рекомендованных

- Иногда применяется, когда продаются достаточно дорогие товары (например, бытовая техника) 

----
Hit rate = (был ли хотя бы 1 релевантный товар среди рекомендованных)   

Hit rate@k = (был ли хотя бы 1 релевантный товар среди топ-k рекомендованных)

In [2]:
def hit_rate(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return (flags.sum() > 0) * 1

def hit_rate_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    return (flags.sum() > 0) * 1

In [3]:
recommended_list = [143, 156, 1134, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]

In [4]:
hit_rate(recommended_list, bought_list)

1

In [5]:
hit_rate_at_k(recommended_list, bought_list, k=3)

1

###  Precision

*Precision* - доля релевантных товаров среди рекомендованных = Какой % рекомендованных товаров  юзер купил

- Пожалуй, самая приближенная к бизнес-метрикам и самая популярная метрика

---

Precision= (# of recommended items that are relevant) / (# of recommended items)  

Precision@k = (# of recommended items @k that are relevant) / (# of recommended items @k)

Money Precision@k = (revenue of recommended items @k that are relevant) / (revenue of recommended items @k)  

**Note:** Обычно k в precision@k достаточно невелико (5-20) и определяется из бизнес-логики. Например, 5 товаров в e-mail рассылке, 20 ответов на первой странице google и т.д

In [6]:
def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)


def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)

def money_precision_at_k_(recommended_list, bought_list, prices_recommended, k=5):
    
    bought_list = np.array(bought_list)
    recommend_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    
    flags = np.isin(recommend_list, bought_list)
    precision = np.dot(flags, prices_recommended).sum() / prices_recommended.sum()
    
    return precision

In [7]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]
prices_recomended = [400, 60, 40, 60 , 90, 90, 240, 200, 50, 30]

In [8]:
precision(recommended_list, bought_list)

0.2

In [9]:
precision_at_k(recommended_list, bought_list, k=3)

0.3333333333333333

In [10]:
money_precision_at_k_(recommended_list, bought_list, prices_recomended, k=5)

0.7076923076923077

###  Recall

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

- Обычно используется для моделей пре-фильтрации товаров (убрать те товары, которые точно не будем рекомендовать)

---

Recall= (# of recommended items that are relevant) / (# of relevant items)  

Recall@k = (# of recommended items @k that are relevant) / (# of relevant items)

Money Recall@k = (revenue of recommended items @k that are relevant) / (revenue of relevant items)  

    
  
**Note:** в recall@k число k обычно достаточно большое (50-200), больше чем покупок у среднестатистического юзера

In [11]:
def recall(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(bought_list)
    

def recall_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(bought_list)


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    
    bought_list = np.array(bought_list)
    prices_bought = np.array(prices_bought)
    recommend_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    
    flags = np.isin(recommend_list, bought_list)
    precision = np.dot(flags, prices_recommended).sum() / prices_bought.sum()
    return precision

In [12]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991, 533]
prices_recomended = [400, 60, 40, 60 , 90, 90, 240, 200, 50, 30]
prices_bought = [100, 200, 400, 60, 200]

In [13]:
recall(recommended_list, bought_list)

0.6

In [14]:
recall_at_k(recommended_list,bought_list,k=5)

0.4

In [15]:
money_recall_at_k(recommended_list, bought_list, prices_recomended, prices_bought, k=10)

0.6875

# Метрики ранжирования

## AP@k
AP@k - average precision at k

$$AP@k = \frac{1}{r} \sum{[recommended_{relevant_i}] * precision@k}$$

- r - кол-во релевантный среди рекомендованных
- Суммируем по всем релевантным товарам
- Зависит от порядка рекомендаций

In [16]:
def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    if len(relevant_indexes) == 0:
        return 0
    
    amount_relevant = len(relevant_indexes)
            
    sum_ = sum([precision_at_k(recommended_list, bought_list, k=index_relevant+1) for index_relevant in relevant_indexes])
    return sum_/amount_relevant

In [17]:
recommended_list = [4,22,21,1,234,232432,234234,666] #id товаров
bought_list = [1,221,3,4,5,6,7,8,9]

In [18]:
ap_k(recommended_list, bought_list, k=5)

0.75

### MAP@k

MAP@k (Mean Average Precision@k)  
Среднее AP@k по всем юзерам
- Показывает средневзвешенную точность рекомендаций

$$MAP@k = \frac{1}{|U|} \sum_u{AP_k}$$
  
|U| - кол-во юзеров

In [19]:
def map_k(recommended_list, bought_list, k=5):
    
    apk_users =[ap_k(recommended_list[user], bought_list[user], k=k) for user in range(len(bought_list_3_users))]
    
    return sum(apk_users)/len(apk_users)

In [20]:
# теперь список из 3 пользователей
recommended_list_3_users = [[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43], 
                    [1134, 533, 14, 4, 15, 1543, 1, 99, 27, 3345],
                    [991, 3345, 27, 533, 43, 143, 1543, 156, 1134, 11]
                    ]

bought_list_3_users = [[521, 32, 143],  # юзер 1
                       [143, 156, 991, 43, 11], # юзер 2
                       [1,2]] # юзер 3

In [21]:
map_k(recommended_list_3_users, bought_list_3_users, k=5)

0.3333333333333333

### Normalized discounted cumulative gain ( NDCG@k)


$$DCG = \frac{1}{|r|} \sum_u{\frac{[bought fact]}{discount(i)}}$$  

$discount(i) = i$ if $i <= 2$,   
$discount(i) = log_2(i+1)$ if $i > 2$


(!) Считаем для первых k рекоммендаций   
(!) - существуют вариации с другими $discount(i)$  
i - ранк рекомендованного товара  
|r| - кол-во рекомендованных товаров 

$$NDCG = \frac{DCG}{ideal DCG}$$

In [22]:
def ndcg_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    flags = np.isin(recommended_list, bought_list)
    
    dcg, ideal_dcg = 0, 0
    for i in range(len(recommended_list)):
        if i <= 1: # нумерация с нуля, уменьшаем условие на 1
            dcg += flags[i] / (i+1)
            ideal_dcg += 1 / (i+1)
        else:
            dcg += flags[i] / (np.log(i+2))
            ideal_dcg += 1 / (np.log(i+2))       
    return dcg / ideal_dcg #   1/(кол-во рек товаров) не учитывается, т.к они сокращаются

In [23]:
recommended_list = [4,22,21,1,234,232432,234234,666] #id товаров
bought_list = [1,221,3,4,5,6,7,8,9]

In [24]:
ndcg_at_k(recommended_list, bought_list, k=5)

0.47675200921606786

### Mean Reciprocal Rank ( MRR@k )


- Считаем для первых k рекоммендаций
- Найти ранк первого релевантного предсказания $k_u$
- Посчитать reciprocal rank = $\frac{1}{k_u}$

$$MRR = mean(\frac{1}{k_u})$$

In [25]:
def min_rank(recommended_list, bought_list, k=1):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    min_rank = np.nonzero(np.isin(recommended_list, bought_list))[0]
    if min_rank.size < 1:
        return 0
    return 1 / (min_rank.min() + 1)

In [26]:
def reciprocal_rank(recommended_list, bought_list, k=1):   
    mrr_k = [min_rank(recommended_list[user], bought_list[user], k=k) for user in range(len(recommended_list))]
    return sum(mrr_k)/len(mrr_k)

In [27]:
reciprocal_rank(recommended_list_3_users, bought_list_3_users, k=5)

0.3333333333333333

### Доп. примеры метрик для оценки рекомендаций/ранжирования (можно взять из интернета, или ваши знания)

- Cumulative Gain at K  В отличии от DCG не нормализована и не учитывает позицию
- Метрики на основе ранговой корреляции (Ранговый коэффициент корреляции Кендэлла, Ранговый коэффициент корреляции Спирмена)
- Метрики на основе каскадной модели поведения (Expected reciprocal rank, PFound)
- WTA (winner takes all) – эта метрика равна 1, если топ-рекомендация (с самым большим предсказанным рейтингом) из просмотренных пользователем полу